In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean , median
import matplotlib.pyplot as plt

# Baseline 

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['Stats Model MAE'].data,nb.scraps['Catboost MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["Stats Model","Catboost"])
baseline_data = np.array(baseLine_data)
stats = median(baseline_data[:,0])
catboost = median(baseline_data[:,1])

# GAN Model

In [4]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEDIAN:")
print(df.median(axis = 0))
gan_data = np.array(gan_data)
gan_median = median(gan_data[:,1])
print(gan_median)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.155729,0.243533,1.764267,4.870652
1,0.480955,0.581771,3.101088,11.635422
2,0.329641,0.404866,2.567177,8.097312
3,0.411285,0.566355,2.867202,11.327104
4,0.252376,0.390330,2.246448,7.806599
5,0.205691,0.367313,2.027688,7.346263
6,0.106401,0.253454,1.458104,5.069089
7,0.268535,0.420745,2.317349,8.414902
8,0.686369,0.581106,3.704330,11.622122
9,0.211097,0.358662,2.054277,7.173240


MEDIAN:
MSE                   0.260455
MAE                   0.397598
Euclidean Distance    2.281899
Manhattan Distance    7.951955
dtype: float64
0.39759777315794487


# ABC_GAN Analysis

## ABC Pre-generator - Catboost 


In [5]:
book = sb.read_notebooks("./ABC_GAN_Catboost")
paramVal = [[1,1],[1,0.1],[1,0.01],[0.1,1],[0.1,0.1],[0.1,0.01],[0.01,1],[0.01,0.1],[0.01,0.01]]
abc_mae = [[] for i in range(9)]
abc_mae_skip = [[] for i in range(9)]
abc_mae_mean = [[] for i in range(9)]
abc_mae_skip_mean = [[] for i in range(9)]
abc_weights = [[] for i in range(9)]
prior_model = [[] for i in range(9)]
abc_pre_generator = [[] for i in range(9)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [6]:

data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,1,0.25488,1.49921,0.95528,0.21571,0.18767
1,1,1,0.30303,1.23703,0.13252,0.17906,0.22285
2,1,1,0.32939,1.49900,0.18995,0.14985,0.14433
3,1,1,0.26696,1.05593,0.66127,0.13549,0.12794
4,1,1,0.25153,1.32671,0.69720,0.12118,0.10116
5,1,1,0.39848,1.06172,0.26413,0.26807,0.19571
6,1,1,0.19024,0.93333,0.12514,0.16389,0.11737
7,1,1,0.39589,1.21225,0.81255,0.21778,0.26979
8,1,1,0.33849,1.19663,0.31259,0.18623,0.17056
9,1,1,0.21904,1.07515,0.15995,0.16206,0.16269


Variance                         1.000000
Bias                             1.000000
Prior Model MAE                  0.284993
ABC pre-generator MAE            1.204442
Skip Node weight                 0.288360
ABC GAN MAE                      0.171475
ABC_GAN MAE (skip connection)    0.166622
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.1,0.34962,1.06319,0.06947,0.28026,0.32690
1,1,0.1,0.20779,1.10523,0.06974,0.17621,0.17678
2,1,0.1,0.22064,1.07093,0.07774,0.16407,0.18702
3,1,0.1,0.29095,0.91358,0.08387,0.20713,0.27912
4,1,0.1,0.30788,1.19335,0.08859,0.14791,0.23631
5,1,0.1,0.33834,0.77644,0.07377,0.26967,0.34508
6,1,0.1,0.23483,1.18687,0.09288,0.19782,0.23874
7,1,0.1,0.26212,1.06847,0.08978,0.17050,0.19537
8,1,0.1,0.35787,1.00405,0.65171,0.32623,0.31562
9,1,0.1,0.22332,1.06554,0.08582,0.18724,0.17234


Variance                         1.000000
Bias                             0.100000
Prior Model MAE                  0.276538
ABC pre-generator MAE            1.067004
Skip Node weight                 0.084845
ABC GAN MAE                      0.192530
ABC_GAN MAE (skip connection)    0.237528
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.01,0.32709,0.84864,0.81082,0.26376,0.27723
1,1,0.01,0.45891,0.81567,0.67028,0.36142,0.38088
2,1,0.01,0.24339,1.07164,0.08196,0.26659,0.24940
3,1,0.01,0.25167,1.04811,0.07212,0.22743,0.23872
4,1,0.01,0.46408,1.19326,0.06925,0.31908,0.43010
5,1,0.01,0.32229,0.87658,0.06997,0.31505,0.30484
6,1,0.01,0.35534,0.89418,0.74588,0.33931,0.29882
7,1,0.01,0.23237,0.94366,0.06718,0.24522,0.21321
8,1,0.01,0.33493,1.05017,0.44877,0.31242,0.30958
9,1,0.01,0.34453,0.94762,0.07980,0.27204,0.33684


...

Variance                         0.010000
Bias                             1.000000
Prior Model MAE                  0.321665
ABC pre-generator MAE            0.911615
Skip Node weight                 0.437501
ABC GAN MAE                      0.197620
ABC_GAN MAE (skip connection)    0.175940
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.1,0.21135,0.21393,0.46296,0.18024,0.19220
1,0.01,0.1,0.28823,0.32711,0.65999,0.23833,0.28216
2,0.01,0.1,0.27976,0.27476,0.54417,0.29839,0.21989
3,0.01,0.1,0.37887,0.38198,0.72997,0.24460,0.22998
4,0.01,0.1,0.22537,0.26056,0.11845,0.15408,0.14684
5,0.01,0.1,0.29931,0.26039,0.00000,0.26314,0.31082
6,0.01,0.1,0.30083,0.32650,0.07792,0.20896,0.28244
7,0.01,0.1,0.23985,0.27859,0.11046,0.20689,0.18685
8,0.01,0.1,0.33576,0.33190,0.32616,0.25361,0.23232
9,0.01,0.1,0.41337,0.42808,0.09309,0.27727,0.26049


Variance                         0.010000
Bias                             0.100000
Prior Model MAE                  0.293772
ABC pre-generator MAE            0.302548
Skip Node weight                 0.222309
ABC GAN MAE                      0.241464
ABC_GAN MAE (skip connection)    0.231152
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.01,0.32289,0.32798,0.01287,0.30408,0.32136
1,0.01,0.01,0.21906,0.22080,0.45020,0.20488,0.21939
2,0.01,0.01,0.26112,0.26000,0.02386,0.19111,0.25670
3,0.01,0.01,0.28543,0.29436,0.00286,0.25346,0.28806
4,0.01,0.01,0.30150,0.30522,0.00000,0.28736,0.30219
5,0.01,0.01,0.38629,0.38339,0.69761,0.34961,0.36539
6,0.01,0.01,0.27426,0.27511,0.55820,0.26998,0.26230
7,0.01,0.01,0.35229,0.34816,0.53971,0.33829,0.33017
8,0.01,0.01,0.25156,0.25473,0.03043,0.23111,0.24712
9,0.01,0.01,0.31986,0.31906,0.02385,0.35149,0.32421


Variance                         0.010000
Bias                             0.010000
Prior Model MAE                  0.293463
ABC pre-generator MAE            0.299790
Skip Node weight                 0.027141
ABC GAN MAE                      0.278670
ABC_GAN MAE (skip connection)    0.295124
dtype: float64
-------------------------------------------------------------


In [7]:
# Display Catboost Summary Tables 
data = np.array(data)
catboostData = []
for i in range(9): 
    catboostData.append([paramVal[i][0], paramVal[i][1],catboost,median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['Variance','Bias','Catboost','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))

,Variance,Bias,Catboost,Prior Model MAE,mGAN,skipGAN,Skip Node weight
0,1.00,1.00,0.15019,1.20444,0.17147,0.16662,0.28836
1,1.00,0.10,0.15019,1.06700,0.19253,0.23753,0.08485
2,1.00,0.01,0.15019,0.94564,0.29223,0.30183,0.08088
3,0.10,1.00,0.15019,0.75765,0.16472,0.14768,0.23734
4,0.10,0.10,0.15019,0.34143,0.25225,0.24970,0.13910
5,0.10,0.01,0.15019,0.33574,0.26668,0.31049,0.04239
6,0.01,1.00,0.15019,0.91162,0.19762,0.17594,0.43750
7,0.01,0.10,0.15019,0.30255,0.24146,0.23115,0.22231
8,0.01,0.01,0.15019,0.29979,0.27867,0.29512,0.02714


## ABC Pre-generator - Stats 


In [8]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [[1,1],[0.1,1],[0.01,1],[1,0.1],[0.1,0.1],[0.01,0.1],[1,0.01],[0.1,0.01],[0.01,0.01]]
abc_mae = [[] for i in range(9)]
abc_mae_skip = [[] for i in range(9)]
abc_mae_mean = [[] for i in range(9)]
abc_mae_skip_mean = [[] for i in range(9)]
abc_weights = [[] for i in range(9)]
prior_model = [[] for i in range(9)]
abc_pre_generator = [[] for i in range(9)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [9]:
data = [[] for i in range(9)]
for i in range(9):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,1,0.40908,1.10139,0.85934,0.20651,0.16391
1,1,1,0.41042,1.32019,0.99024,0.27560,0.22850
2,1,1,0.34189,1.38493,0.99243,0.28316,0.17152
3,1,1,0.34599,1.28583,0.88857,0.29835,0.15427
4,1,1,0.40760,1.46145,0.98863,0.31588,0.23272
5,1,1,0.26292,1.16470,0.91238,0.26853,0.18664
6,1,1,0.41536,1.17383,0.90924,0.33878,0.15214
7,1,1,0.46712,1.32771,0.97800,0.48451,0.22742
8,1,1,0.40423,1.32115,0.98528,0.30436,0.26602
9,1,1,0.40066,1.14204,1.00000,0.25613,0.17187


Variance                         1.000000
Bias                             1.000000
Prior Model MAE                  0.405913
ABC pre-generator MAE            1.303008
Skip Node weight                 0.981644
ABC GAN MAE                      0.290755
ABC_GAN MAE (skip connection)    0.179254
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,1,0.52136,0.81170,0.93736,0.29945,0.19836
1,0.1,1,0.42667,0.88454,0.88077,0.26522,0.20568
2,0.1,1,0.40904,0.83525,0.94681,0.40484,0.20381
3,0.1,1,0.49473,1.10486,0.88486,0.23727,0.19190
4,0.1,1,0.35824,0.81270,0.97526,0.21981,0.15826
5,0.1,1,0.39750,1.12945,0.98278,0.24304,0.20855
6,0.1,1,0.31279,1.09576,0.99090,0.23434,0.23928
7,0.1,1,0.39661,0.83775,1.00000,0.22590,0.19992
8,0.1,1,0.49191,1.02374,0.89998,0.32193,0.22931
9,0.1,1,0.46810,0.72521,0.99496,0.25526,0.16161


Variance                         0.100000
Bias                             1.000000
Prior Model MAE                  0.417857
ABC pre-generator MAE            0.861147
Skip Node weight                 0.961035
ABC GAN MAE                      0.249152
ABC_GAN MAE (skip connection)    0.201866
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,1,0.34208,1.06281,0.96790,0.19264,0.16789
1,0.01,1,0.34047,0.82213,0.98897,0.29801,0.23198
2,0.01,1,0.33042,1.05162,0.91036,0.21778,0.10698
3,0.01,1,0.30579,0.91307,0.99857,0.16342,0.21422
4,0.01,1,0.38413,0.92709,0.96132,0.27958,0.20624
5,0.01,1,0.52283,1.05918,0.99273,0.25753,0.27060
6,0.01,1,0.43137,0.95676,0.90589,0.31383,0.16247
7,0.01,1,0.46726,0.76921,0.99332,0.28664,0.33421
8,0.01,1,0.31540,0.71590,0.99084,0.18193,0.12482
9,0.01,1,0.48232,0.82943,0.96845,0.23853,0.22870


...

Variance                         1.000000
Bias                             0.010000
Prior Model MAE                  0.374747
ABC pre-generator MAE            0.980198
Skip Node weight                 0.317261
ABC GAN MAE                      0.287311
ABC_GAN MAE (skip connection)    0.233179
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.01,0.49230,0.51399,0.12942,0.38151,28.27809
1,0.1,0.01,0.29982,0.31046,0.27293,0.29066,62.90161
2,0.1,0.01,0.41310,0.40431,0.07710,0.30654,226.53303
3,0.1,0.01,0.28925,0.32271,0.16547,0.23204,0.24526
4,0.1,0.01,0.45239,0.40022,0.20453,0.33694,64.43114
5,0.1,0.01,0.35190,0.35466,0.09987,0.23345,0.23204
6,0.1,0.01,0.37064,0.38155,0.13743,0.48735,0.32087
7,0.1,0.01,0.29918,0.31110,0.07976,0.37793,19.46445
8,0.1,0.01,0.35718,0.34842,0.09866,0.20500,33.84229
9,0.1,0.01,0.45058,0.45452,0.10034,0.27653,0.18642


Variance                          0.100000
Bias                              0.010000
Prior Model MAE                   0.363909
ABC pre-generator MAE             0.368106
Skip Node weight                  0.114881
ABC GAN MAE                       0.298600
ABC_GAN MAE (skip connection)    23.871272
dtype: float64
-------------------------------------------------------------


,Variance,Bias,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.01,0.45341,0.45326,0.14017,0.35608,0.23465
1,0.01,0.01,0.34609,0.34193,0.10421,0.22331,0.20010
2,0.01,0.01,0.40232,0.39803,0.18738,0.42377,0.36342
3,0.01,0.01,0.32400,0.32628,0.14579,0.34244,105.98985
4,0.01,0.01,0.36699,0.36494,0.16711,0.20662,0.28224
5,0.01,0.01,0.39431,0.39236,0.22245,0.40595,0.22446
6,0.01,0.01,0.44717,0.44629,0.11131,0.37020,0.28247
7,0.01,0.01,0.34248,0.34579,0.09757,0.38565,10.01225
8,0.01,0.01,0.47569,0.47295,0.15139,0.31280,645.21505
9,0.01,0.01,0.46516,0.46412,0.13947,0.37827,0.38524


Variance                         0.010000
Bias                             0.010000
Prior Model MAE                  0.398316
ABC pre-generator MAE            0.395198
Skip Node weight                 0.142977
ABC GAN MAE                      0.363143
ABC_GAN MAE (skip connection)    0.322945
dtype: float64
-------------------------------------------------------------


In [10]:
# Display Stats Summary Tables 
data = np.array(data)
catboostData = []
for i in range(9): 
    catboostData.append([paramVal[i][0], paramVal[i][1],stats,median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(catboostData, columns = ['Variance','Bias','Stats','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))

,Variance,Bias,Stats,Prior Model MAE,mGAN,skipGAN,Skip Node weight
0,1.00,1.00,0.37376,1.30301,0.29076,0.17925,0.98164
1,0.10,1.00,0.37376,0.86115,0.24915,0.20187,0.96103
2,0.01,1.00,0.37376,0.92008,0.24803,0.21023,0.97871
3,1.00,0.10,0.37376,0.96247,0.29560,0.17623,0.38288
4,0.10,0.10,0.37376,0.39296,0.29301,0.23849,0.15733
5,0.01,0.10,0.37376,0.42667,0.29443,0.16880,0.19300
6,1.00,0.01,0.37376,0.98020,0.28731,0.23318,0.31726
7,0.10,0.01,0.37376,0.36811,0.29860,23.87127,0.11488
8,0.01,0.01,0.37376,0.39520,0.36314,0.32295,0.14298
